# DGIM Implementation

The DGIM algorithm is used to count the number of "True" (1) present in a stream of 1/0 considering only the last N elements. N elements essentially constitute the sliding window. The implementation is done using a compact data structure that consumes $O(log^2(N))$ space. Explanation can be found in Chapter 4 of the book.

In the initialize function below. Intialize the following:
1. Set the error rate. Make sure that the error rate is in [0, 1]. Let c be the actual result and e be the result generated by the implmentation. Then $abs(c-e) < error\_rate * c$.

2. Now set the number of buckets (b) to be maintained in the algorithm. Set $b=1/error\_rate$. Check that the maximum number of buckets of the same size is 2.

3. The data structure to hold the buckets can be an array of queues. queue[i] will hold the timestamp of the bucket in descending order. This helps in updating the buckets and also knowing at any point of time that how many buckets of the same size there is.
        a. So, first intialize an array.
        b. Now, find out the maximum number of buckets that can be needed which is log(N) (base 2).
        c. For values computed above, initialize 1 queue each.
4. Set two variables with currentTimeStamp and oldestTimeStamp


1.   Timestamp
      1.   log2n bits

2.   Bucket
      1.   timestamp of end
      2.   No. of 1's between start and end

3.   Non Overlapping, Sorted, Maximum 2 buckets of same size.



In [0]:
import math
from collections import deque
N = 0 #sliding window width
error_rate = 0
buckets = 0
max_index = 0
timestamp = 0
oldest_bucket_timestamp = -1
# to access the global variables use global variable_name
def initialize(n, error = 0.5):
    global N,error_rate,buckets,max_index,timestamp,oldest_bucket_timestamp
    N = n
    error_rate = error
    # buckets = math.ceil(1/error_rate)
    buckets = 2
    if N == 0:
        max_index = -1
    else:
        max_index = int(math.ceil(math.log(N)/math.log(2)))

    

    timestamp = 0
    oldest_bucket_timestamp = -1
 


Now write the update function for adding a new element by doing the following:

1. If the new element is 1. (Do nothing for 0)
    1. Update the currentTimeStamp with a maximum value of 2*N
    2. Check if the oldest bucket needs to be removed by checking if it is too old. If it is then delete that.
3. Update the oldestTimeStamp with currentTimeStamp
4. For all the elements in the queue do the following:
    1. Add the new element
    2. pop the last two elements from the queue.
    3. Merge last two buckets if needed

In [0]:
def bucket_too_old(b_timestamp):
    global N,error_rate,buckets,max_index,timestamp,oldest_bucket_timestamp,queues
    if (timestamp - b_timestamp) %(2*N) >= N:
        return True
    
    return False

def delete_oldest_bucket():
    global N,error_rate,buckets,max_index,timestamp,oldest_bucket_timestamp,queues
    for q in reversed(queues):
        if len(q) > 0:
            q.pop()
            break
    oldest_bucket_timestamp = -1
    for q in reversed(queues):
        if len(q) > 0:
            oldest_bucket_timestamp = q[-1]
            break

def update(element):
    global N,error_rate,buckets,max_index,timestamp,oldest_bucket_timestamp,queues
    if N == 0:
        return
    
    timestamp = (timestamp + 1)%(2*N)
    if(oldest_bucket_timestamp >= 0 and bucket_too_old(oldest_bucket_timestamp)):
        delete_oldest_bucket()
    
    if element == 0:
        return
    
    carry = timestamp
    if oldest_bucket_timestamp == -1:
        oldest_bucket_timestamp = timestamp
    for q in queues:
        q.appendleft(carry)
        if len(q) <= buckets:
            break
        last = q.pop()
        s_last = q.pop()
        
        carry = s_last
        if last == oldest_bucket_timestamp:
            oldest_bucket_timestamp = s_last
            
            

        

Now write the count function to count the number of elements in N and return the estimate of the number of 1s.
1. For all the queues:
    1. Find queue length
    2. For every non-null queue increment count variable with the length times the power of two (starting from 1).
2. Return half of the count computed in the step above.

In [0]:
def count():
    global queues
    ans = 0
    m = 0
    p = 1
    for q in queues:
        l = len(q)
        if l > 0:
            m = p
            ans += l*p
        
        p = 2*p
        
    ans -= math.floor(m/2)
    
    return int(ans)

You can write some auxilliary functions to deleteOldestBucket() and checkIfBucketTooOld()

Now make a call to initialize, make some updates and check the result that you get.

In [4]:
initialize(100, 0.1)   
queues = [deque() for _ in range(max_index + 1)]

for i in range(N):
#     if i%2 == 0:
#         update(1)
#     else:
#         update(0)
    update(1)
    # print(queues)
    
c = count()

print('Total number of ones in window size of ',N,' : ',end=' ')
print(c)

print('Error : ',(N-c)/N)

Total number of ones in window size of  100  :  84
Error :  0.16
